In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import sys
sys.path.append('/Users/vincentchen/code/metal')
import metal
import torch
from torch.utils.data import Dataset, DataLoader

In [3]:
print('PyTorch: ', torch.__version__)
print('MeTaL:   ', metal.__version__)
print('Python:  ', sys.version)
print('Python:  ', sys.version_info)

PyTorch:  0.4.1
MeTaL:    0.3.3
Python:   3.6.7 | packaged by conda-forge | (default, Nov 20 2018, 18:20:05) 
[GCC 4.2.1 Compatible Apple LLVM 9.0.0 (clang-900.0.37)]
Python:   sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)


In [9]:
from metal.contrib.slicing.sqlite_wrapper import SnorkelDataset
import os

db_conn_str   = os.path.join(os.getcwd(),"spouses.db")
# candidate_def = ['ChemicalDisease', ['chemical', 'disease']]
# candidate_def = candidate_subclass('Spouse', ['person1', 'person2'])
candidate_def = ['Spouse', ['person1', 'person2']]


train, dev, test = SnorkelDataset.splits(db_conn_str, 
                                         candidate_def, 
                                         max_seq_len=125)

print(f'[TRAIN] {len(train)}')
print(f'[DEV]   {len(dev)}')
print(f'[TEST]  {len(test)}')

Connected to sqlite:////Users/vincentchen/code/metal/metal/contrib/slicing/Spouses/spouses.db
Connected to sqlite:////Users/vincentchen/code/metal/metal/contrib/slicing/Spouses/spouses.db
Connected to sqlite:////Users/vincentchen/code/metal/metal/contrib/slicing/Spouses/spouses.db
[TRAIN] 22254
[DEV]   2811
[TEST]  2701


In [8]:
import numpy as np
snorkel_data = np.load('snorkel_data_spouse.npz')
L_train = snorkel_data['L_train']
L_dev = snorkel_data['L_dev']
L_test = snorkel_data['L_test']
train_marginals = snorkel_data['train_marginals']
dev_marginals = snorkel_data['dev_marginals']
accs = snorkel_data['accs']
m = len(accs)

L_train.shape, L_dev.shape, L_test.shape, len(train_marginals), len(dev_marginals)

((22254, 10), (2811, 10), (2701, 10), 22254, 2811)

In [17]:
train[0][0].shape

torch.Size([125])

In [13]:
from metal.contrib.slicing.CDR.embeddings import EmbeddingLoader, load_embeddings
emb_path  = "../glove.6B/glove.6B.50d.txt"
embs  = EmbeddingLoader(emb_path, fmt='text')

In [20]:
from metal.end_model import EndModel
from metal.modules import LSTMModule
use_cuda = torch.cuda.is_available()

wembs = load_embeddings(train.word_dict, embs)
lstm = LSTMModule(embed_size=50, 
                  hidden_size=50, 
                  embeddings=wembs,
                  lstm_reduction='attention', 
                  dropout=0.25, 
                  num_layers=1, 
                  freeze=False)

oracle = EndModel([100, 2], input_module=lstm, seed=123, use_cuda=use_cuda)
oracle.config['train_config']['optimizer_config']['optimizer_common']['lr'] = 0.01
oracle.config['train_config']['validation_metric'] = 'f1'
oracle.config['train_config']['batch_size'] = 32
oracle.config['train_config']['n_epochs'] = 10

%time oracle.train_model(train, dev_data=dev)
oracle.score(test, metric=['precision', 'recall', 'f1'])

Loaded 91.0% (29001/31870) pretrained embeddings
Using pretrained embeddings.
Embeddings shape = (31870, 50)
The embeddings are NOT FROZEN
Using lstm_reduction = 'attention'

Network architecture:
Sequential(
  (0): Sequential(
    (0): LSTMModule(
      (embeddings): Embedding(31870, 50)
      (lstm): LSTM(50, 50, batch_first=True, bidirectional=True)
    )
    (1): ReLU()
  )
  (1): Linear(in_features=100, out_features=2, bias=True)
)



Process Process-7:
Traceback (most recent call last):
  File "/Users/vincentchen/miniconda3/envs/slicing/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/vincentchen/miniconda3/envs/slicing/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

  File "/Users/vincentchen/miniconda3/envs/slicing/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/Users/vincentchen/miniconda3/envs/slicing/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/Users/vincentchen/miniconda3/envs/slicing/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/Users/vincentchen/miniconda3/envs/slicing/lib/python3.

Traceback (most recent call last):
  File "/Users/vincentchen/miniconda3/envs/slicing/lib/python3.6/site-packages/IPython/core/magics/execution.py", line 1263, in time
    out = eval(code, glob, local_ns)
  File "<timed eval>", line 1, in <module>
  File "/Users/vincentchen/code/metal/metal/end_model/end_model.py", line 226, in train_model
    train_loader, loss_fn, dev_data=dev_data, log_writer=log_writer
  File "/Users/vincentchen/code/metal/metal/classifier.py", line 200, in _train_model
    for batch_num, data in t:
  File "/Users/vincentchen/miniconda3/envs/slicing/lib/python3.6/site-packages/tqdm/_tqdm_notebook.py", line 221, in __iter__
    for obj in super(tqdm_notebook, self).__iter__(*args, **kwargs):
  File "/Users/vincentchen/miniconda3/envs/slicing/lib/python3.6/site-packages/tqdm/_tqdm.py", line 979, in __iter__
    for obj in iterable:
  File "/Users/vincentchen/miniconda3/envs/slicing/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 330, in __next__
   

KeyboardInterrupt: 

Process Process-8:
Traceback (most recent call last):
  File "/Users/vincentchen/miniconda3/envs/slicing/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/vincentchen/miniconda3/envs/slicing/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/vincentchen/miniconda3/envs/slicing/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/Users/vincentchen/miniconda3/envs/slicing/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/Users/vincentchen/miniconda3/envs/slicing/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/Users/vincentchen/miniconda3/envs/slicing/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/Users/vincentc

KeyboardInterrupt: 